<a href="https://colab.research.google.com/github/AbdullahMalik17/Agentic_AI/blob/main/Copy_of_OPENAI_SDK_OPENROUTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [2]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00


In [3]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [4]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [5]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "mistralai/mistral-small-24b-instruct-2501:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [6]:
BASE_URL

'https://openrouter.ai/api/v1'

In [7]:
import requests
import json

response = requests.post(
  url=f"https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "Hi, i have 1000 pkr i want you to convert into usd"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1751627800-7uNRnalEkhpmIOnBfIly', 'provider': 'Chutes', 'model': 'mistralai/mistral-small-24b-instruct-2501:free', 'object': 'chat.completion', 'created': 1751627800, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': "The conversion rate between Pakistani Rupees (PKR) and United States Dollars (USD) can fluctuate, so the current exchange rate will determine the exact amount in USD.\n\nAs of my last update in October 2023, I can't provide real-time exchange rates, but you can check a reliable financial website or use a currency converter to get the most accurate and up-to-date conversion rate.\n\nFor instance, if the exchange rate is 1 USD = 250 PKR (just an example rate), you would calculate:\n\n1000 PKR / 250 PKR/USD = 4 USD\n\nAgain, please verify the current exchange rate for an accurate conversion.", 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 23, 'complet

In [8]:
data = response.json()
data['choices'][0]['message']['content']

"The conversion rate between Pakistani Rupees (PKR) and United States Dollars (USD) can fluctuate, so the current exchange rate will determine the exact amount in USD.\n\nAs of my last update in October 2023, I can't provide real-time exchange rates, but you can check a reliable financial website or use a currency converter to get the most accurate and up-to-date conversion rate.\n\nFor instance, if the exchange rate is 1 USD = 250 PKR (just an example rate), you would calculate:\n\n1000 PKR / 250 PKR/USD = 4 USD\n\nAgain, please verify the current exchange rate for an accurate conversion."

## 2. Using OpenAI Agents SDK

In [9]:
import asyncio
from openai import AsyncOpenAI # chat completions
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

set_tracing_disabled(disabled=True) # Open AI Tracing == Disable

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)


async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="LahoreTA",
        instructions="You only respond in english in multi lines",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent, # starting agent
        "What is your name?. Are you chatgpt model", # request
    )
    print(result.final_output)


asyncio.run(main())

I don't have a name. I am a machine learning model designed to understand and generate text based on the input I receive. I am not the same model as ChatGPT, but I am designed to assist and engage in conversation to the best of my ability.

How can I assist you today? If you have any questions or need information on a specific topic, feel free to ask.


In [10]:
response.json()

{'id': 'gen-1751627800-7uNRnalEkhpmIOnBfIly',
 'provider': 'Chutes',
 'model': 'mistralai/mistral-small-24b-instruct-2501:free',
 'object': 'chat.completion',
 'created': 1751627800,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant',
    'content': "The conversion rate between Pakistani Rupees (PKR) and United States Dollars (USD) can fluctuate, so the current exchange rate will determine the exact amount in USD.\n\nAs of my last update in October 2023, I can't provide real-time exchange rates, but you can check a reliable financial website or use a currency converter to get the most accurate and up-to-date conversion rate.\n\nFor instance, if the exchange rate is 1 USD = 250 PKR (just an example rate), you would calculate:\n\n1000 PKR / 250 PKR/USD = 4 USD\n\nAgain, please verify the current exchange rate for an accurate conversion.",
    'refusal': None,
    'reasoning': None}}],
 'usage': 